In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/videogamesales/vgsales.csv


In [2]:
game_file_path = '/kaggle/input/videogamesales/vgsales.csv'
features_data = pd.read_csv(game_file_path, index_col='Rank')
features_data.columns

Index(['Name', 'Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'],
      dtype='object')

In [3]:
features_data.shape

(16598, 10)

In [4]:
features_data.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [5]:
y = features_data.Global_Sales

In [6]:
features_data.drop('Name', axis=1, inplace=True)

In [7]:
## Shape of training data (num_rows, num_columns)
print(features_data.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (features_data.isnull().sum())
print(missing_val_count_by_column)
print(100 * missing_val_count_by_column/features_data.shape[0])

(16598, 9)
Platform          0
Year            271
Genre             0
Publisher        58
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64
Platform        0.000000
Year            1.632727
Genre           0.000000
Publisher       0.349440
NA_Sales        0.000000
EU_Sales        0.000000
JP_Sales        0.000000
Other_Sales     0.000000
Global_Sales    0.000000
dtype: float64


In [8]:
features_data.dropna(subset=['Year'], inplace=True)
features_data.dropna(subset=['Publisher'], inplace=True)

In [9]:
# Get list of categorical variables
s = (features_data.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['Platform', 'Genre', 'Publisher']


In [10]:
print(features_data.Platform.unique())
print("")
print(features_data.Genre.unique())
print("")
print(features_data.Publisher.unique())

['Wii' 'NES' 'GB' 'DS' 'X360' 'PS3' 'PS2' 'SNES' 'GBA' '3DS' 'PS4' 'N64'
 'PS' 'XB' 'PC' '2600' 'PSP' 'XOne' 'GC' 'WiiU' 'GEN' 'DC' 'PSV' 'SAT'
 'SCD' 'WS' 'NG' 'TG16' '3DO' 'GG' 'PCFX']

['Sports' 'Platform' 'Racing' 'Role-Playing' 'Puzzle' 'Misc' 'Shooter'
 'Simulation' 'Action' 'Fighting' 'Adventure' 'Strategy']

['Nintendo' 'Microsoft Game Studios' 'Take-Two Interactive'
 'Sony Computer Entertainment' 'Activision' 'Ubisoft' 'Bethesda Softworks'
 'Electronic Arts' 'Sega' 'SquareSoft' 'Atari' '505 Games' 'Capcom'
 'GT Interactive' 'Konami Digital Entertainment'
 'Sony Computer Entertainment Europe' 'Square Enix' 'LucasArts'
 'Virgin Interactive' 'Warner Bros. Interactive Entertainment'
 'Universal Interactive' 'Eidos Interactive' 'RedOctane' 'Vivendi Games'
 'Enix Corporation' 'Namco Bandai Games' 'Palcom' 'Hasbro Interactive'
 'THQ' 'Fox Interactive' 'Acclaim Entertainment' 'MTV Games'
 'Disney Interactive Studios' 'Majesco Entertainment' 'Codemasters'
 'Red Orb' 'Level 5' 'Arena En

In [11]:
from sklearn.preprocessing import OrdinalEncoder

# Make copy to avoid changing original data 
label_features_data = features_data.copy()

# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
label_features_data[object_cols] = ordinal_encoder.fit_transform(features_data[object_cols])

In [12]:
print("Category to Label Mapping:")
print(ordinal_encoder.categories_)

Category to Label Mapping:
[array(['2600', '3DO', '3DS', 'DC', 'DS', 'GB', 'GBA', 'GC', 'GEN', 'GG',
       'N64', 'NES', 'NG', 'PC', 'PCFX', 'PS', 'PS2', 'PS3', 'PS4', 'PSP',
       'PSV', 'SAT', 'SCD', 'SNES', 'TG16', 'WS', 'Wii', 'WiiU', 'X360',
       'XB', 'XOne'], dtype=object), array(['Action', 'Adventure', 'Fighting', 'Misc', 'Platform', 'Puzzle',
       'Racing', 'Role-Playing', 'Shooter', 'Simulation', 'Sports',
       'Strategy'], dtype=object), array(['10TACLE Studios', '1C Company', '20th Century Fox Video Games',
       '2D Boy', '3DO', '49Games', '505 Games', '5pb', '7G//AMES',
       '989 Sports', '989 Studios', 'AQ Interactive', 'ASC Games',
       'ASCII Entertainment', 'ASCII Media Works', 'ASK', 'Abylight',
       'Acclaim Entertainment', 'Accolade', 'Ackkstudios', 'Acquire',
       'Activision', 'Activision Blizzard', 'Activision Value',
       'Adeline Software', 'Aerosoft', 'Agatsuma Entertainment', 'Agetec',
       'Aksys Games', 'Alawar Entertainment', 'Alchemi

In [13]:
label_features_data.shape

(16291, 9)

In [14]:
label_features_data.columns

Index(['Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales',
       'JP_Sales', 'Other_Sales', 'Global_Sales'],
      dtype='object')

In [15]:
print(label_features_data.Platform.unique())
print("")
print(label_features_data.Genre.unique())
print("")
print(label_features_data.Publisher.unique())

[26. 11.  5.  4. 28. 17. 16. 23.  6.  2. 18. 10. 15. 29. 13.  0. 19. 30.
  7. 27.  8.  3. 20. 21. 22. 25. 12. 24.  1.  9. 14.]

[10.  4.  6.  7.  5.  3.  8.  9.  0.  2.  1. 11.]

[359. 323. 493. 455.  21. 524.  66. 138. 445. 465.  53.   6.  85. 177.
 275. 457. 464. 288. 540. 547. 527. 137. 426. 543. 144. 347. 385. 214.
 487. 164.  17. 292. 126. 296.  91. 424. 281.  40. 325. 253. 120.  10.
 343. 521. 391. 308. 237. 499. 532.  13. 330. 241. 528. 462. 531.  23.
  63. 109. 378. 425. 537. 219. 198. 199. 550. 206. 101. 224.  94. 430.
  55. 485.  12. 565.  18. 459.   4. 419. 348. 160.  30.  71. 460. 303.
 372. 564. 413.  88. 142. 472.  97. 434. 490.  27. 176. 322. 400. 450.
 511. 441. 273. 182. 508. 299. 191. 334. 398.  84. 495. 104. 436. 414.
 574. 420. 525. 443. 246. 341.  87. 463.   2.  58. 223. 367. 314. 365.
  67. 513. 461. 256. 362. 277. 415. 233. 506. 170. 432. 318. 305. 227.
 433. 456. 526. 193. 552.  61. 381. 515. 279. 232. 491. 263. 122. 146.
 557. 238. 481.  83.  44. 186. 324. 255.

In [16]:
label_features_data.isnull().sum()

Platform        0
Year            0
Genre           0
Publisher       0
NA_Sales        0
EU_Sales        0
JP_Sales        0
Other_Sales     0
Global_Sales    0
dtype: int64

In [17]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()
imputed_features_data = pd.DataFrame(my_imputer.fit_transform(label_features_data))

imputed_features_data.columns = label_features_data.columns

In [18]:
imputed_features_data.columns

Index(['Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales',
       'JP_Sales', 'Other_Sales', 'Global_Sales'],
      dtype='object')

In [19]:
missing_val_count_by_column = (imputed_features_data.isnull().sum())
print(missing_val_count_by_column)

Platform        0
Year            0
Genre           0
Publisher       0
NA_Sales        0
EU_Sales        0
JP_Sales        0
Other_Sales     0
Global_Sales    0
dtype: int64


In [20]:
features = ['Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales', 
            'JP_Sales', 'Other_Sales']

X = imputed_features_data[features]

y = imputed_features_data.Global_Sales

print(imputed_features_data.describe)

<bound method NDFrame.describe of        Platform    Year  Genre  Publisher  NA_Sales  EU_Sales  JP_Sales  \
0          26.0  2006.0   10.0      359.0     41.49     29.02      3.77   
1          11.0  1985.0    4.0      359.0     29.08      3.58      6.81   
2          26.0  2008.0    6.0      359.0     15.85     12.88      3.79   
3          26.0  2009.0   10.0      359.0     15.75     11.01      3.28   
4           5.0  1996.0    7.0      359.0     11.27      8.89     10.22   
...         ...     ...    ...        ...       ...       ...       ...   
16286       6.0  2002.0    4.0      269.0      0.01      0.00      0.00   
16287       7.0  2003.0    8.0      241.0      0.01      0.00      0.00   
16288      16.0  2008.0    6.0       21.0      0.00      0.00      0.00   
16289       4.0  2010.0    5.0        8.0      0.00      0.01      0.00   
16290       6.0  2003.0    4.0      544.0      0.01      0.00      0.00   

       Other_Sales  Global_Sales  
0             8.46         82.

In [21]:
"""
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=50, max_depth=2, random_state=1)
imputed_X_train = fit_transform(X_train)
imputed_X_valid = lab.fit_transform(X_valid)
rf_model.fit(imputed_X_train, y_train)

rf_valid_predictions = rf_model.predict(imputed_X_valid)
"""

'\nfrom sklearn.ensemble import RandomForestClassifier\nrf_model = RandomForestClassifier(n_estimators=50, max_depth=2, random_state=1)\nimputed_X_train = fit_transform(X_train)\nimputed_X_valid = lab.fit_transform(X_valid)\nrf_model.fit(imputed_X_train, y_train)\n\nrf_valid_predictions = rf_model.predict(imputed_X_valid)\n'